In [1]:
import polars as pl
from datetime import datetime

In [2]:
df = pl.read_csv("./data/healthcare_dataset.csv")

In [4]:
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

In [6]:
required_columns = ['name', 'age', 'gender', 'blood_type']

In [10]:
df = df.unique()

In [11]:
df = (
df.with_columns(
    pl.concat_str(
        *[pl.col(col) for col in required_columns]
    )
    .hash()
    .cast(pl.String)
    .alias('patient_id')
))
       

df = df.select(['patient_id'] + [col for col in df.columns if col != 'patient_id'])

In [12]:
df.get_column('patient_id').is_duplicated().sum()

0